# SetUp

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os . chdir ( "/content/gdrive/My Drive/Colab Notebooks/5) Deep Learning/Projects/Common/Sign Board Text Detection")

In [ ]:
%reset-f

In [ ]:
train_model = True

In [ ]:
import torch
from torch import nn
# import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

import random
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm_notebook

# Import only when we need to train model
# Remove all files from folder named -> 'Object Detection Files' <- and paste in main folder
if train_model :
    from engine import train_one_epoch, evaluate
    import utils
    import transforms as T

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Read Dataset

In [ ]:
def return_cleaned_text(word, language) :
    final_word = ''
    word = word.upper()
    if language == 'eng' :
        for letter in word :
            if letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' : final_word += letter
    else :
        for letter in word :
            if letter in 'ऀँंःऄअआइईउऊऋऌऍऎएऐऑऒओऔकखगघङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसहऺऻ़ऽािीुूृॄॅॆेैॉॊोौ्ॎॏॐ॒॑॓॔ॕॖॗक़ख़ग़ज़ड़ढ़फ़य़ॠॡॢॣ०१२३४५६७८९॰ॱॲॳॴॵॶॷॸॹॺॻॼॽॾॿঀঁংঃ' :
                final_word += letter
    return final_word

# return_cleaned_text('राsdfghसवि हारी|(?  विहा', language='hin')
# return_cleaned_text('RAAS AVहारी|(?IHA  AREE.', language='eng')

In [ ]:
english_words = []
hindi_words = []
all_words_dict = {}

file1 = open("Dataset/Transliteration/Train.txt","r")
for line in file1.readlines()[1:-1] :
    if 'Source' in line : english_words.append(line[12:-14])
    if 'Target' in line and len(english_words)>len(hindi_words) : hindi_words.append(line[19:-14])
file1.close()
for eng_word, hin_word in zip(english_words, hindi_words) :
    if ' ' in hin_word : hin_word = hin_word.split()
    if ' ' in eng_word : eng_word = eng_word.split()
    if type(hin_word)==list or type(eng_word)==list :
        hin_word, eng_word = list(hin_word), list(eng_word)
        if len(hin_word) == len(eng_word) :
            for each_hin_word, each_eng_word in zip(hin_word, eng_word) :
                all_words_dict[return_cleaned_text(each_hin_word, language='hin')] = return_cleaned_text(each_eng_word, language='eng')
    else : all_words_dict[return_cleaned_text(hin_word, language='hin')] = return_cleaned_text(eng_word, language='eng')

file1 = open("Dataset/Transliteration/Dataset - 2 .txt","r")
for line in file1.readlines() :
    eng_word, hin_word = line.split('\t')
    all_words_dict[return_cleaned_text(hin_word, language='hin')] = return_cleaned_text(eng_word, language='eng')
file1.close()

print(all_words_dict)
print(len(all_words_dict))

{'रासविहारी': 'RASVIHARI', 'देवगन': 'DEVGAN', 'रोड': 'ROAD', 'शत्रुमर्दन': 'SHATRUMARDAN', 'महिजुबा': 'MAHIJUBA', 'सैबिन': 'SAIBIN', 'बिल': 'BILL', 'कॉस्बी': 'COSBY', 'रिश्ता': 'RISTA', 'कागज़': 'KAGAZ', 'का': 'KA', 'हातिम': 'HATIM', 'श्रीमयी': 'SRIMAYI', 'फरीहाह': 'FARIHAH', 'मैरीटाइम': 'MARITIME', 'म्युज़ियम': 'MUSIUM', 'ऑफ': 'OF', 'ग्रीस': 'GREECE', 'मंथन': 'MANTHAN', 'फ्रेंकोरशियन': 'FRANCORUSSIAN', 'वार': 'BAR', 'तन्मया': 'TANMYA', 'मल्ली': 'MALLI', 'केलीमुटु': 'KELIMUTU', 'मुटाटकर': 'MUTATAKAR', 'गंगा': 'GANGA', 'मैया': 'MAIYA', 'फरीदाह': 'FARIDAH', 'तहमीना': 'TAHMEENA', 'दुर्रानी': 'DURANII', 'डान्यूब': 'DANUBE', 'बलील': 'BALEEL', 'डाल्मिया': 'DALMIYA', 'सीमेंट': 'CEMENT', 'फ्लोरस': 'FLORUS', 'मोंटगोमेरी': 'MONTGOMERY', 'विद्या': 'VIDYA', 'सारंगी': 'SAARANGEE', 'लवीना': 'LAVINA', 'अरबाना': 'ARBANA', 'संधर': 'SANDHAR', 'महारानी': 'MAHARANI', 'पद्मिनी': 'PADMINI', 'ब्रिटा': 'BRITA', 'जिरोफ्ट': 'JIROFT', 'नाग': 'NAAG', 'देवता': 'DEVTA', 'वेरका': 'VERKA', 'छपरा': 'CHAPRA', 'कचेरी': '

In [ ]:
for num in range(1, 19) :
    file_name = 'File - ' + str(num) + ' (transliterated).txt'
    with open('Dataset/Transliteration/'+file_name) as file:
        for line in file.readlines() :
            hindi_word, english_word = line[:-1].split('\t')
            all_words_dict[return_cleaned_text(hindi_word, language='hin')] = return_cleaned_text(english_word, language='eng')

all_words_dict['यश'] = 'YASH'
len(all_words_dict)

14880

In [ ]:
english_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '<pad>'

english_alphabets_dict = { pad_char:0, '<sos>':1, '<eos>':2 }
for index, alphabet in enumerate(english_alphabets) :
    english_alphabets_dict[alphabet] = index + 3
print(english_alphabets_dict)
print(len(english_alphabets_dict))

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6, 'E': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26, 'Y': 27, 'Z': 28}
29


In [ ]:
# Hindi Unicode Hex Range is 2304:2435

pad_char = '<pad>'
hindi_alphabets_dict = { pad_char:0, '<sos>':1, '<eos>':2 }
for num in range(2304, 2436) :
    hindi_alphabets_dict[chr(num)] = num - 2301
print(hindi_alphabets_dict)
print(len(hindi_alphabets_dict))

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ऀ': 3, 'ँ': 4, 'ं': 5, 'ः': 6, 'ऄ': 7, 'अ': 8, 'आ': 9, 'इ': 10, 'ई': 11, 'उ': 12, 'ऊ': 13, 'ऋ': 14, 'ऌ': 15, 'ऍ': 16, 'ऎ': 17, 'ए': 18, 'ऐ': 19, 'ऑ': 20, 'ऒ': 21, 'ओ': 22, 'औ': 23, 'क': 24, 'ख': 25, 'ग': 26, 'घ': 27, 'ङ': 28, 'च': 29, 'छ': 30, 'ज': 31, 'झ': 32, 'ञ': 33, 'ट': 34, 'ठ': 35, 'ड': 36, 'ढ': 37, 'ण': 38, 'त': 39, 'थ': 40, 'द': 41, 'ध': 42, 'न': 43, 'ऩ': 44, 'प': 45, 'फ': 46, 'ब': 47, 'भ': 48, 'म': 49, 'य': 50, 'र': 51, 'ऱ': 52, 'ल': 53, 'ळ': 54, 'ऴ': 55, 'व': 56, 'श': 57, 'ष': 58, 'स': 59, 'ह': 60, 'ऺ': 61, 'ऻ': 62, '़': 63, 'ऽ': 64, 'ा': 65, 'ि': 66, 'ी': 67, 'ु': 68, 'ू': 69, 'ृ': 70, 'ॄ': 71, 'ॅ': 72, 'ॆ': 73, 'े': 74, 'ै': 75, 'ॉ': 76, 'ॊ': 77, 'ो': 78, 'ौ': 79, '्': 80, 'ॎ': 81, 'ॏ': 82, 'ॐ': 83, '॑': 84, '॒': 85, '॓': 86, '॔': 87, 'ॕ': 88, 'ॖ': 89, 'ॗ': 90, 'क़': 91, 'ख़': 92, 'ग़': 93, 'ज़': 94, 'ड़': 95, 'ढ़': 96, 'फ़': 97, 'य़': 98, 'ॠ': 99, 'ॡ': 100, 'ॢ': 101, 'ॣ': 102, '।': 103, '॥': 104, '०': 105, '१': 106, '२': 107, '३': 108, '४': 10

In [ ]:
# Check if all letters are present

for word in all_words_dict :
    hin_word, eng_word = word, all_words_dict[word]
    for letter in hin_word :
        if letter not in hindi_alphabets_dict.keys() : print('hin ----> ', letter, ' ------------------- ', ord(letter))
    for letter in eng_word :
        if letter not in english_alphabets_dict.keys() : print('eng ----> ', letter, ' ------------------- ', ord(letter))

In [ ]:
num_data_points = 14880

def create_subset_of_main_dataset (dict1, num_samples) :
    final_dict = {}
    for i in range(num_samples) :
        hindi_word = list(dict1.keys())[i]
        english_word = dict1[hindi_word]
        final_dict[hindi_word] = english_word
    return final_dict

all_words_dict = create_subset_of_main_dataset(all_words_dict, num_data_points)
print(all_words_dict)
len(all_words_dict)

{'रासविहारी': 'RASVIHARI', 'देवगन': 'DEVGAN', 'रोड': 'ROAD', 'शत्रुमर्दन': 'SHATRUMARDAN', 'महिजुबा': 'MAHIJUBA', 'सैबिन': 'SAIBIN', 'बिल': 'BILL', 'कॉस्बी': 'COSBY', 'रिश्ता': 'RISTA', 'कागज़': 'KAGAZ', 'का': 'KA', 'हातिम': 'HATIM', 'श्रीमयी': 'SRIMAYI', 'फरीहाह': 'FARIHAH', 'मैरीटाइम': 'MARITIME', 'म्युज़ियम': 'MUSIUM', 'ऑफ': 'OF', 'ग्रीस': 'GREECE', 'मंथन': 'MANTHAN', 'फ्रेंकोरशियन': 'FRANCORUSSIAN', 'वार': 'BAR', 'तन्मया': 'TANMYA', 'मल्ली': 'MALLI', 'केलीमुटु': 'KELIMUTU', 'मुटाटकर': 'MUTATAKAR', 'गंगा': 'GANGA', 'मैया': 'MAIYA', 'फरीदाह': 'FARIDAH', 'तहमीना': 'TAHMEENA', 'दुर्रानी': 'DURANII', 'डान्यूब': 'DANUBE', 'बलील': 'BALEEL', 'डाल्मिया': 'DALMIYA', 'सीमेंट': 'CEMENT', 'फ्लोरस': 'FLORUS', 'मोंटगोमेरी': 'MONTGOMERY', 'विद्या': 'VIDYA', 'सारंगी': 'SAARANGEE', 'लवीना': 'LAVINA', 'अरबाना': 'ARBANA', 'संधर': 'SANDHAR', 'महारानी': 'MAHARANI', 'पद्मिनी': 'PADMINI', 'ब्रिटा': 'BRITA', 'जिरोफ्ट': 'JIROFT', 'नाग': 'NAAG', 'देवता': 'DEVTA', 'वेरका': 'VERKA', 'छपरा': 'CHAPRA', 'कचेरी': '

14880

# Create Dataset Class & DataLoader

In [ ]:
class text_processing (torch.utils.data.Dataset) :

    def __init__ (self, word_dictionary, english_alphabets_dict, hindi_alphabets_dict) :
        self.word_dictionary = word_dictionary
        self.english_alphabets_dict = english_alphabets_dict
        self.hindi_alphabets_dict = hindi_alphabets_dict

    def __len__ (self) : return len(self.word_dictionary)

    def get_word_vector_with_padding (self, word, dict1) :
        word = word[:14]
        encode_word = np.array((list([1] + [0]*(len(word)) + [2] + [0]*(14-len(word)))), dtype=float)
        for i, letter in enumerate(word) : encode_word[i+1] = float(dict1[letter])
        return np.array(encode_word, dtype=float)

    def __getitem__ (self, index) :
        hindi_word = list(self.word_dictionary.keys())[index]
        english_word = self.word_dictionary[hindi_word]
        hindi_vector = self.get_word_vector_with_padding(hindi_word, self.hindi_alphabets_dict)
        english_vector = self.get_word_vector_with_padding(english_word, self.english_alphabets_dict)
        return torch.LongTensor(hindi_vector), torch.LongTensor(english_vector)

In [ ]:
dataset = text_processing(word_dictionary=all_words_dict, english_alphabets_dict=english_alphabets_dict, hindi_alphabets_dict=hindi_alphabets_dict)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)

dataset[1]

(tensor([ 1, 41, 74, 56, 26, 43,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([ 1,  6,  7, 24,  9,  3, 16,  2,  0,  0,  0,  0,  0,  0,  0,  0]))

In [ ]:
dataiter = iter(data_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([16, 16])
torch.Size([16, 16])


In [ ]:
print(type(images))
images

<class 'torch.Tensor'>


tensor([[ 1, 18, 49, 65, 36, 78, 51,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 47, 65, 41, 47, 65, 43,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 39, 59, 80, 43, 67, 49,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 43, 65, 51,  5, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 43, 31, 53, 65,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 49, 68, 46, 67, 41,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 24, 80, 50, 68, 49, 66,  5, 26,  2,  0,  0,  0,  0,  0,  0],
        [ 1, 59, 26, 67, 43, 65,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 47, 66, 43, 78, 41, 66, 43, 67,  2,  0,  0,  0,  0,  0,  0],
        [ 1, 34, 74, 53, 80, 50, 69, 51, 66, 50, 49,  2,  0,  0,  0,  0],
        [ 1, 12, 45, 65, 57,  5, 40, 65,  2,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 56, 74, 53, 80, 60, 75, 49,  2,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 59, 74,  5, 29, 69, 51, 66, 50, 43,  2,  0,  0,  0,  0,  0],
        [ 1, 47, 66, 36, 63, 53, 65,  

# Transformers - Architecture (Classes & Functions)

In [ ]:
# Transliteration from hindi to english

# Hindi word length (with padding) -> 16

# After embeddings -> 16 * 512

### Self - Attention

In [ ]:
class self_attention (nn.Module) :

    def __init__ (self, embedding_size=512, num_heads=8) :
        super(self_attention, self).__init__()

        self.embedding_size = embedding_size                                    # embedding_size = 512 (paper)
        self.num_heads = num_heads                                              # num_heads = 8 (paper)
        self.head_dimensions = embedding_size // num_heads                      # head_dimensions = 64

        assert self.head_dimensions * num_heads == embedding_size, "Embed size needs to be divisible by the number of heads"

        self.keys_matrix = nn.Linear(self.head_dimensions, self.head_dimensions, bias=False)                            # keys_matrix -> (64, 64)
        self.queries_matrix = nn.Linear(self.head_dimensions, self.head_dimensions, bias=False)                         # queries_matrix -> (64, 64)
        self.values_matrix = nn.Linear(self.head_dimensions, self.head_dimensions, bias=False)                          # values_matrix -> (64, 64)

        self.final_matrix = nn.Linear(self.head_dimensions * self.num_heads, self.embedding_size)                       # final_matrix -> (512, 512)

    def forward (self, keys, queries, values, mask=None) :                      # Input -> (N, 16, 512)

        self.keys = keys                                                        # keys -> (N, 16, 512)
        self.queries = queries                                                  # queries -> (N, 16, 512)
        self.values = values                                                    # values -> (N, 16, 512)

        self.N = queries.shape[0]                                               # self.N -> N

        self.keys_length = self.keys.shape[1]                                   # keys_length = 16
        self.queries_length = self.queries.shape[1]                             # queries_length = 16
        self.values_length = self.values.shape[1]                               # values_length = 16

        self.keys = self.keys.reshape(self.N, self.keys_length, self.num_heads, self.head_dimensions)                   # keys -> (N, 16, 8, 64)
        self.queries = self.queries.reshape(self.N, self.queries_length, self.num_heads, self.head_dimensions)          # queries -> (N, 16, 8, 64)
        self.values = self.values.reshape(self.N, self.values_length, self.num_heads, self.head_dimensions)             # values -> (N, 16, 8, 64)

        self.keys = self.keys_matrix(self.keys)                                 # keys -> (N, 16, 8, 64) x (64, 64) -> (N, 16, 8, 64) <no change in shape>
        self.queries = self.queries_matrix(self.queries)                        # queries -> (N, 16, 8, 64) x (64, 64) -> (N, 16, 8, 64) <no change in shape>
        self.values = self.values_matrix(self.values)                           # values -> (N, 16, 8, 64) x (64, 64) -> (N, 16, 8, 64) <no change in shape>

        self.keys_cross_prod_queries = torch.einsum('nqhd,nkhd->nhqk', self.queries, self.keys)                         # (N, 16, 8, 64) x (N, 16, 8, 64) -> (N, 8, 16, 16)
                                                                                # Here, we multiply the queries and keys to get attention matrix between the words
                                                                                # So we need an output of sentence_length * sentence_length as it shows how related
                                                                                #       each word in with the other words in the sentence
                                                                                # Each of the 8 heads generates a matrix that shows this relation and so we have
                                                                                #       8 such matrices

        if mask is not None : self.keys_cross_prod_queries = self.keys_cross_prod_queries.masked_fill(mask == 0, float("-1e20"))

        self.keys_cross_prod_queries = torch.softmax(self.keys_cross_prod_queries/(self.embedding_size**(1/2)), dim=3)

        self.attention = torch.einsum('nhql,nlhd->nqhd', self.keys_cross_prod_queries, self.values).reshape(self.N, self.queries_length, self.num_heads * self.head_dimensions)
        # self.attention = torch.einsum('nhqk,nvhd->nqhd', self.keys_cross_prod_queries, self.values).reshape(self.N, self.queries_length, self.num_heads * self.head_dimensions)
                                                                                # (Keys * Queries) x Values -> (N, 8, 16, 16) x (N, 16, 8, 64) -> (N, 16, 8, 64)
                                                                                # (N, 16, 8, 64) -> Reshape -> (N, 16, 8 * 64) -> (N, 16, 512)
                                                                                # In the required shape to return

        self.attention = self.final_matrix(self.attention)                      # (N, 16, 512) x (512, 512) -> (N, 16, 512)

        return self.attention                                                   # Output -> (N, 16, 512)

### Encoder

In [ ]:
# Used as the repeated cell

class encoder_cell (nn.Module) :

    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(encoder_cell, self).__init__()

        self.self_attention_block = self_attention(embedding_size, num_heads_in_attention_cell)
        self.normalize = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.feed_forward_block = nn.Sequential (                                                                   # Input -> (16, 512)
                                                nn.Linear(embedding_size, embedding_size*expanssion_size) ,        # (16, 512) x (512, 1024) -> (16, 1024)
                                                nn.ReLU() ,
                                                nn.Linear(embedding_size*expanssion_size, embedding_size) ,        # (16, 1024) x (1024, 512) -> (16, 512)
                                                )                                                                  # Return -> (16, 512)

    def add_and_concat_operation (self, matrix1, matrix2) : return self.dropout(self.normalize(matrix1 + matrix2))

    def forward (self, keys, queries, values, mask=None) :
        attention = self.self_attention_block(keys, queries, values, mask)
        attention = self.add_and_concat_operation(attention, queries)
        feed_forward_output = self.feed_forward_block(attention)
        feed_forward_output = self.add_and_concat_operation(feed_forward_output, attention)
        return feed_forward_output

In [ ]:
# Complete encoder - with repeated cells and positional encodings

class Encoder (nn.Module) :

    def __init__ (self, sorce_vocab_size, device='cpu', max_input_length=16, embedding_size=512,
                  num_heads_in_attention_cell=8, expanssion_size=2, num_encoder_layers=6, dropout=0) :
        super(Encoder, self).__init__()

        self.embedding_size = embedding_size
        self.device = device
        self.dropout = nn.Dropout(dropout)
        self.word_embedding = nn.Embedding(sorce_vocab_size, embedding_size)
        self.positional_embedding = nn.Embedding(max_input_length, embedding_size)

        self.layers = nn.ModuleList([encoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout).to(device) for i in range(num_encoder_layers)])

    def forward (self, word_vector, mask=None) :

        N, word_length = word_vector.shape
        word_vector.to(device)
        position_vector = torch.arange(0, word_length).expand(N, word_length).to(self.device)
        print(position_vector)
        print(word_vector)
        output = self.dropout(self.word_embedding(word_vector) + self.positional_embedding(position_vector))

        for layer in self.layers : output = layer(output, output, output, mask)

        return output

    def forward(self, x, mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.positional_embedding(positions))
        for layer in self.layers:
            out = layer(out, out, out, mask)
        return out

### Decoder

In [ ]:
# Part of repeated decoder section - the upper half is the same as the encoder section
# Same as encoder part

class sub_decoder_cell (nn.Module) :

    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(sub_decoder_cell, self).__init__()

        self.self_attention_block = self_attention(embedding_size, num_heads_in_attention_cell)
        self.normalize = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.feed_forward_block = nn.Sequential(                                                                   # Input -> (16, 512)
                                                nn.Linear(embedding_size, embedding_size*expanssion_size) ,        # (16, 512) x (512, 1024) -> (16, 1024)
                                                nn.ReLU() ,
                                                nn.Linear(embedding_size*expanssion_size, embedding_size) ,        # (16, 1024) x (1024, 512) -> (16, 512)
                                                )                                                                  # Return -> (16, 512)

    def add_and_concat_operation (self, matrix1, matrix2) : return self.dropout(self.normalize(matrix1 + matrix2))

    def forward (self, keys, queries, values, mask=None) :
        attention = self.self_attention_block(keys, queries, values, mask)
        attention = self.add_and_concat_operation(attention, queries)
        feed_forward_output = self.feed_forward_block(attention)
        feed_forward_output = self.add_and_concat_operation(feed_forward_output, attention)
        return feed_forward_output

In [ ]:
# Used as the repeated decoder cell
# This is the complete decoder part without embeddings

class decoder_cell (nn.Module) :

    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(decoder_cell, self).__init__()

        self.self_attention_block = self_attention(embedding_size, num_heads_in_attention_cell)
        self.normalize = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.sub_decoder_cell = sub_decoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout)

    def add_and_concat_operation (self, matrix1, matrix2) : return self.dropout(self.normalize(matrix1 + matrix2))

    def forward (self, keys, queries, values, source_mask=None, target_mask=None) :
        # For the masked multi-headed attention block
        # Only queries input comes to decoder
        # Keys and Values are input from encoder
        attention = self.self_attention_block(queries, queries, queries, target_mask)
        queries = self.add_and_concat_operation(attention, queries)

        # This is part of the sub_decoder_cell in which keys & values are output from encoder
        # This block is same as the encoder part
        output = self.sub_decoder_cell(keys, queries, values, source_mask)
        return output

In [ ]:
class Decoder (nn.Module) :

    def __init__ (self, target_vocab_size, embedding_size=512, num_decoder_layers=6, num_heads_in_attention_cell=8,
                  max_output_length=16, expanssion_size=2, dropout=0, device='cpu') :
        super(Decoder, self).__init__()

        self.embedding_size = embedding_size
        self.device = device

        self.word_embeddings = nn.Embedding(target_vocab_size, embedding_size)
        self.positional_embedding = nn.Embedding(max_output_length, embedding_size)

        self.layers = nn.ModuleList([decoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout) for _ in range(num_decoder_layers)])
        self.dropout = nn.Dropout(dropout)
        self.FC_layer = nn.Linear(embedding_size, target_vocab_size)

    def forward (self, keys, queries, values, source_mask=None, target_mask=None) :

        N, word_length = queries.shape
        positions = torch.arange(0, word_length).expand(N, word_length).to(self.device)
        # queries = torch.tensor(queries, dtype=int).clone().detach()
        output = self.dropout(self.word_embeddings(queries) + self.positional_embedding(positions))

        for layer in self.layers :
            output = layer.forward(keys, output, values, source_mask, target_mask)

        output = self.FC_layer(output)
        return output

### Transformer

In [ ]:
class Transformer (nn.Module) :

    def __init__ (self, source_vocab_size, target_vocab_size, source_pad_index=0, target_pad_index=0, embedding_size=512, num_encoder_layers=6, num_decoder_layers=6,
                  num_heads_in_attention_cell=8, expanssion_size=2, dropout=0., max_input_length=16, max_output_length=16, device='cpu') :
        super(Transformer, self).__init__()

        self.encoder = Encoder(source_vocab_size, device, max_input_length, embedding_size, num_heads_in_attention_cell, expanssion_size, num_encoder_layers, dropout)
        self.decoder = Decoder(target_vocab_size, embedding_size, num_decoder_layers, num_heads_in_attention_cell, max_output_length, expanssion_size, dropout, device)

        self.source_pad_index = source_pad_index
        self.target_pad_index = target_pad_index
        self.device = device

    def make_source_mask (self, source_vector) :
        source_mask = (source_vector != self.source_pad_index).unsqueeze(1).unsqueeze(2)
        return source_mask.to(self.device)

    def make_target_mask (self, target_vector) :
        N, target_length = target_vector.shape
        target_mask = torch.tril(torch.ones(target_length, target_length)).expand(N, 1, target_length, target_length)
        return target_mask.to(self.device)

    def forward (self, source_vector, target_vector) :
        source_mask = self.make_source_mask(source_vector)
        target_mask = self.make_target_mask(target_vector)

        encoder_output = self.encoder(source_vector, source_mask)
        decoder_output = self.decoder(keys=encoder_output, queries=target_vector, values=encoder_output, source_mask=source_mask, target_mask=target_mask)

        return decoder_output

# Training

In [ ]:
def output_english_word (hindi_word, hindi_dict, english_dict, model, max_output_length=16, device=device, verbose=True) :
    predictions = [1]
    english_word = ''
    hindi_word = return_cleaned_text(hindi_word, language='hin')[:14]
    hindi_word_vector = np.array([1] + [0]*len(hindi_word) + [2] + [0]*(14-len(hindi_word)))
    for i, letter in enumerate(hindi_word) : hindi_word_vector[i+1] = hindi_dict[letter]
    hindi_word_vector = torch.tensor(np.array(hindi_word_vector).reshape(1,-1))
    for i in range(max_output_length) :
        input_tensor = torch.tensor(predictions).unsqueeze(0).to(device)
        with torch.no_grad() : out = model(hindi_word_vector.to(device), input_tensor.to(device))
        word_index = out.argmax(dim=-1)[:, -1].item()
        predictions.append(word_index)
        if word_index == 2 : break
        else : english_word += english_dict[word_index]
    if verbose : print(hindi_word, ' ------> ', english_word)
    return predictions, hindi_word

In [ ]:
dataset = text_processing(word_dictionary=all_words_dict, english_alphabets_dict=english_alphabets_dict, hindi_alphabets_dict=hindi_alphabets_dict)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)

In [ ]:
model = Transformer(source_vocab_size=len(hindi_alphabets_dict), target_vocab_size=len(english_alphabets_dict), source_pad_index=0, target_pad_index=0,
                          embedding_size=512, num_encoder_layers=6, num_decoder_layers=6, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.,
                          max_input_length=16, max_output_length=16, device=device).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss(ignore_index=0)
opt = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
print(all_words_dict)
len(all_words_dict)

{'रासविहारी': 'RASVIHARI', 'देवगन': 'DEVGAN', 'रोड': 'ROAD', 'शत्रुमर्दन': 'SHATRUMARDAN', 'महिजुबा': 'MAHIJUBA', 'सैबिन': 'SAIBIN', 'बिल': 'BILL', 'कॉस्बी': 'COSBY', 'रिश्ता': 'RISTA', 'कागज़': 'KAGAZ', 'का': 'KA', 'हातिम': 'HATIM', 'श्रीमयी': 'SRIMAYI', 'फरीहाह': 'FARIHAH', 'मैरीटाइम': 'MARITIME', 'म्युज़ियम': 'MUSIUM', 'ऑफ': 'OF', 'ग्रीस': 'GREECE', 'मंथन': 'MANTHAN', 'फ्रेंकोरशियन': 'FRANCORUSSIAN', 'वार': 'BAR', 'तन्मया': 'TANMYA', 'मल्ली': 'MALLI', 'केलीमुटु': 'KELIMUTU', 'मुटाटकर': 'MUTATAKAR', 'गंगा': 'GANGA', 'मैया': 'MAIYA', 'फरीदाह': 'FARIDAH', 'तहमीना': 'TAHMEENA', 'दुर्रानी': 'DURANII', 'डान्यूब': 'DANUBE', 'बलील': 'BALEEL', 'डाल्मिया': 'DALMIYA', 'सीमेंट': 'CEMENT', 'फ्लोरस': 'FLORUS', 'मोंटगोमेरी': 'MONTGOMERY', 'विद्या': 'VIDYA', 'सारंगी': 'SAARANGEE', 'लवीना': 'LAVINA', 'अरबाना': 'ARBANA', 'संधर': 'SANDHAR', 'महारानी': 'MAHARANI', 'पद्मिनी': 'PADMINI', 'ब्रिटा': 'BRITA', 'जिरोफ्ट': 'JIROFT', 'नाग': 'NAAG', 'देवता': 'DEVTA', 'वेरका': 'VERKA', 'छपरा': 'CHAPRA', 'कचेरी': '

14880

In [ ]:
english_reverse_dict = {}
for keys in english_alphabets_dict :
    english_reverse_dict[english_alphabets_dict[keys]] = keys

print(english_alphabets_dict)
print(english_reverse_dict)

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6, 'E': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26, 'Y': 27, 'Z': 28}
{0: '<pad>', 1: '<sos>', 2: '<eos>', 3: 'A', 4: 'B', 5: 'C', 6: 'D', 7: 'E', 8: 'F', 9: 'G', 10: 'H', 11: 'I', 12: 'J', 13: 'K', 14: 'L', 15: 'M', 16: 'N', 17: 'O', 18: 'P', 19: 'Q', 20: 'R', 21: 'S', 22: 'T', 23: 'U', 24: 'V', 25: 'W', 26: 'X', 27: 'Y', 28: 'Z'}


In [ ]:
num = len(all_words_dict) // 2
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

देवपुरा  ------>  IIIIIISISZRIRIIG


([1, 11, 11, 11, 11, 11, 11, 21, 11, 21, 28, 20, 11, 20, 11, 11, 9], 'देवपुरा')

In [ ]:
num = len(all_words_dict) // 3
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

मनसा  ------>  AWIIIISIGIIIGI<pad>I


([1, 3, 25, 11, 11, 11, 11, 21, 11, 9, 11, 11, 11, 9, 11, 0, 11], 'मनसा')

In [ ]:
num = len(all_words_dict) // 4
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

शिल्पा  ------>  IWIIIJWISZGIRI<pad>G


([1, 11, 25, 11, 11, 11, 12, 25, 11, 21, 28, 9, 11, 20, 11, 0, 9], 'शिल्पा')

In [ ]:
num = len(all_words_dict) // 5
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

ट्रम्प  ------>  IGIIGJWIGIIIRI<pad>G


([1, 11, 9, 11, 11, 9, 12, 25, 11, 9, 11, 11, 11, 20, 11, 0, 9], 'ट्रम्प')

### Main Training Loop

In [ ]:
%%time

max_epochs = 8
print_epoch_num = 30
save_model = True
path = '/content/gdrive/My Drive/Colab Notebooks/5) Deep Learning/Projects/Common/Sign Board Text Detection/Models - CheckPoints/Transliteration/'

least_loss = 10000

for epoch in tqdm_notebook(range(max_epochs), total=max_epochs, unit="epoch"):
    print('Epoch number -> ', epoch)
    for i, data in enumerate(data_loader, 0):

        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()
        outputs = model(inputs, labels[:, :-1])
        outputs = outputs.reshape(-1, outputs.shape[2])

        labels = labels[:, 1:].reshape(-1)
        loss = loss_fn(outputs, labels)

        curr_loss = loss.item()

        if ((i+1)%print_epoch_num==0) : print('                Loss value at batch ', i+1, ' -----------> ', curr_loss)

        if curr_loss < least_loss :
            least_loss = curr_loss
            print('                                ---------------- Saving model with loss = ', curr_loss, ' ---------------- ')
            if save_model : torch.save(model.state_dict(), path + 'Final Code - Best Model Mid Epoch (epoch = ' + str(epoch+1) + ').pt')

        loss.backward()
        opt.step()
    print('Completed Epoch number -> ', epoch)
    if save_model and epoch>(max_epochs/2) : torch.save(model.state_dict(), path + 'Final Code - End of Epoch ' + str(epoch+1) + '.pt')
    print('\n-------------------------------------------------------------------\n')

if save_model : torch.save(model.state_dict(), path + 'Complete Model.pt')

  0%|          | 0/8 [00:00<?, ?epoch/s]

Epoch number ->  0
                                ---------------- Saving model with loss =  3.58364200592041  ---------------- 
                                ---------------- Saving model with loss =  3.3120243549346924  ---------------- 
                                ---------------- Saving model with loss =  3.3104662895202637  ---------------- 
                                ---------------- Saving model with loss =  3.0898311138153076  ---------------- 
                                ---------------- Saving model with loss =  3.030702590942383  ---------------- 
                                ---------------- Saving model with loss =  2.840301990509033  ---------------- 
                                ---------------- Saving model with loss =  2.8263609409332275  ---------------- 
                                ---------------- Saving model with loss =  2.764906167984009  ---------------- 
                                ---------------- Saving model with loss =  2.71626

In [ ]:
# CPU : 10min 8s ; epochs = 1
# GPU : 10min 5s ; epochs = 8

In [ ]:
# Data

# num_samples = 32 ; batches = 2 ; epochs = 128 ; final_loss = 0.011234564706683159 ; time = 2min 4s
# num_samples = 64 ; batches = 4 ; epochs = 64 ; final_loss = 0.02568313665688038 ; time = 1min 46s
# num_samples = 128 ; batches = 8 ; epochs = 32 ; final_loss = 0.09787848591804504 ; time = 1min 40s
# num_samples = 256 ; batches = 16 ; epochs = 16 ; final_loss = 0.7773730754852295 ; time = 1min 36s
# num_samples = 512 ; batches = 32 ; epochs = 8 ; final_loss = 1.2657017707824707 ; time = 1min 34s
# num_samples = 1024 ; batches = 64 ; epochs = 8 ; final_loss = 0.8635595440864563 ; time = 3min 4s
# num_samples = 2048 ; batches = 128 ; epochs = 8 ; final_loss = 0.7626952528953552 ; time = 6min 6s
# num_samples = 4096 ; batches = 256 ; epochs = 8 ; final_loss = 0.8631321787834167 ; time = 12min 12s
# num_samples = 8192 ; batches = 512 ; epochs = 8 ; final_loss = 0.3729851245880127 ; time = 24min 35s
# num_samples = 10240 ; batches = 640 ; epochs = 8 ; final_loss = 0.604170024394989 ; time = 30min 18s
# num_samples = 12288 ; batches = 768 ; epochs = 8 ; final_loss = 0.688862919807434 ; time = 36min 13s
# num_samples = 13984 ; batches = 874 ; epochs = 8 ; final_loss = 0.2888166308403015 ; time = 42min 9s
# num_samples = 14880 ; batches = 930 ; epochs = 8 ; final_loss = 0.5692203044891357 ; time = 43min 23s

In [ ]:
num = len(all_words_dict) // 2
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

देवपुरा  ------>  DEVPURA


([1, 6, 7, 24, 18, 23, 20, 3, 2], 'देवपुरा')

In [ ]:
num = len(all_words_dict) // 3
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

मनसा  ------>  MANSA


([1, 15, 3, 16, 21, 3, 2], 'मनसा')

In [ ]:
num = len(all_words_dict) // 4
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

शिल्पा  ------>  SHILPA


([1, 21, 10, 11, 14, 18, 3, 2], 'शिल्पा')

In [ ]:
num = len(all_words_dict) // 5
output_english_word(list(all_words_dict.keys())[num], hindi_alphabets_dict, english_reverse_dict, model)

ट्रम्प  ------>  TRUMP


([1, 22, 20, 23, 15, 18, 2], 'ट्रम्प')

### Testing Model Performance - Random Inputs

In [ ]:
for i in range(2,40) :
    num = len(all_words_dict) // i
    ele = output_english_word(list(all_words_dict.keys())[num-1], hindi_alphabets_dict, english_reverse_dict, model)
    print()

फिरोज़पुर  ------>  FIROZPUR

बेलसाउथ  ------>  BELSOUTH

गड़वाल  ------>  GARWAL

उपाध्ये  ------>  UPADHYE

फै़क  ------>  FAINK

रेल्वे  ------>  RELVE

पूरब  ------>  POORAB

मुकुट  ------>  MUKUT

राजमहल  ------>  RAJMAHAL

बलसाड  ------>  BALSAD

टेम्पलो  ------>  TEMPLO

शादी  ------>  SHADI

एकेसिया  ------>  EKESIYA

मुल्तानी  ------>  MULTANI

जेनिबेल  ------>  JENIBEL

नॉर्थ  ------>  NORTH

अतीफा  ------>  ATIFA

टॉमस  ------>  TOMAS

अवार्ड  ------>  AWARD

आचार्यसुता  ------>  ACHARYASUTA

बनारहट  ------>  BANARHAT

दो  ------>  DO

यलकूत  ------>  YALKUT

डबेंग्वा  ------>  DABENGWA

क्यूरेन  ------>  KYUREN

टेक्नोलॉजी  ------>  TECHNOLOGY

नटवर  ------>  NATVAR

अजमल  ------>  AJMAL

नेगी  ------>  NEGI

एट्टा  ------>  ETTA

विनोथन  ------>  VINOTHAN

स्वाति  ------>  SWATI

फिर  ------>  FIR

ट्रंप  ------>  TRUNP

शॉना  ------>  SHANA

क्राउन  ------>  CROWN

डलसाइमर  ------>  DALSIMER

परेरा  ------>  PARERA



In [ ]:
text1 = 'का कक्षा तीसरी में पढ़ता था उसके पास तीन छोटे प्यारे प्यारे खरगोश थे रितेश अपने खरगोश को बहुत प्यार करता था वह स्कूल जाने से पहले पाक से हरे भरे कोमल घास लाकर अपने खरगोश को खिलाता था और फिर स्कूल जाता था स्कूल से आकर भी उसके लिए घास लाता था एक दिन की बात है रितेश को स्कूल के लिए देरी हो रही थी वह घास नहीं ला सका और स्कूल चला गया जब स्कूल से आया तो खरगोश अपने घर में नहीं था रितेश ने खूब ढूंढा परंतु कहीं नहीं मिला सब लोगों से पूछा मगर खरगोश कहीं भी नहीं मिला रितेश उदास हो गया रो रोकर आंखें लाल हो गई रितेश अब पार्क में बैठ कर रोने लगा कुछ देर बाद वह देखता है कि उसके तीनों खरगोश घास खा रहे थे और खेल रहे थे रितेश को खुशी हुई और वह समझ गया कि इन को भूख लगी थी इसलिए यह पार्क में आए हैं मुझे भूख लगती है तो मैं मां से खाना मांग लेता हूं पर इनकी तो मैं भी नहीं है उसे दुख भी हुआ और खरगोश को मिलने की खुशी हुई'
text1 = text1.split()

for hindi_word in text1 :
    hindi_word = return_cleaned_text(hindi_word, language='hin')
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

का  ------>  KA

कक्षा  ------>  KASSHA

तीसरी  ------>  TIISARII

में  ------>  MEN

पढ़ता  ------>  PADHTA

था  ------>  THA

उसके  ------>  USKE

पास  ------>  PAAS

तीन  ------>  TEEN

छोटे  ------>  CHOTE

प्यारे  ------>  PYARE

प्यारे  ------>  PYARE

खरगोश  ------>  KHARAGOSH

थे  ------>  THE

रितेश  ------>  RITESH

अपने  ------>  APNE

खरगोश  ------>  KHARAGOSH

को  ------>  KO

बहुत  ------>  BAHUT

प्यार  ------>  PYAAR

करता  ------>  KARTA

था  ------>  THA

वह  ------>  WAH

स्कूल  ------>  SCUL

जाने  ------>  JAANE

से  ------>  SEY

पहले  ------>  PAHALE

पाक  ------>  PAAK

से  ------>  SEY

हरे  ------>  HARE

भरे  ------>  BHARE

कोमल  ------>  KOMAL

घास  ------>  GHAAS

लाकर  ------>  LAKAR

अपने  ------>  APNE

खरगोश  ------>  KHARAGOSH

को  ------>  KO

खिलाता  ------>  KHILAATA

था  ------>  THA

और  ------>  AUR

फिर  ------>  FIR

स्कूल  ------>  SCUL

जाता  ------>  JAATA

था  ------>  THA

स्कूल  ------>  SCUL

से  ------>  SEY

आकर  ------>  AAKAR

भी  -

In [ ]:
hindi_word = 'यश'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

यश  ------>  YASH


In [ ]:
hindi_word = 'शिखर'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

शिखर  ------>  SHIKHAR


In [ ]:
hindi_word = 'अनुपम'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

अनुपम  ------>  ANUPAM


In [ ]:
hindi_word = 'रश्मि'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

रश्मि  ------>  RASHMI


In [ ]:
hindi_word = 'प्रकाश'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

प्रकाश  ------>  PRAKASH


In [ ]:
hindi_word = 'भवेश'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

भवेश  ------>  BHAVESH


In [ ]:
hindi_word = 'भ्रष्टाचार'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

भ्रष्टाचार  ------>  BHRASTACHAR


In [ ]:
hindi_word = 'सुधारना'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

सुधारना  ------>  SUDHARANA


In [ ]:
hindi_word = 'दिक्कत'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

दिक्कत  ------>  DIKKAT


In [ ]:
hindi_word = 'व्यक्त'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

व्यक्त  ------>  VYAKT


In [ ]:
hindi_word = 'सुझाव '
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

सुझाव  ------>  SUJHAAV


In [ ]:
hindi_word = 'शिकायत'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

शिकायत  ------>  SHIKAYAT


In [ ]:
hindi_word = 'निर्भर'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

निर्भर  ------>  NIRBHAR


In [ ]:
hindi_word = 'सुधारना'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

सुधारना  ------>  SUDHARANA


In [ ]:
hindi_word = 'आथमन'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

आथमन  ------>  AATHAMAN


In [ ]:
hindi_word = 'सोनी'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

सोनी  ------>  SONI


In [ ]:
hindi_word = 'प्रेस्टीन'
ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)

प्रेस्टीन  ------>  PRESTIN


In [ ]:
line = 'उद्यान निर्विकार उद्धघोषित निष्क्रिय तराना निलम्बित समान्तर विश्लेषण सन्मुख उद्देश्य'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

उद्यान  ------>  UDYAN

निर्विकार  ------>  NIRVIKAR

उद्धघोषित  ------>  UDDHAGHOSHTH

निष्क्रिय  ------>  NISHKARIY

तराना  ------>  TARANA

निलम्बित  ------>  NILMBIT

समान्तर  ------>  SAMANTAR

विश्लेषण  ------>  VISHLESHAN

सन्मुख  ------>  SANMUKH

उद्देश्य  ------>  UDDESHYA



In [ ]:
line = 'अनौपचारिक सहृदय अन्वेषरा साक्षात्कार उत्तरद्यिकारी प्रतिबिम्ब आनुवंशिक अभिव्यक्ति'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

अनौपचारिक  ------>  ANOUPCHARIC

सहृदय  ------>  SAHRIDAY

अन्वेषरा  ------>  ANWESHARA

साक्षात्कार  ------>  SAKSATKA

उत्तरद्यिकारी  ------>  UTTARADIYAKARI

प्रतिबिम्ब  ------>  PRATIBIMB

आनुवंशिक  ------>  ANUVANSHIK

अभिव्यक्ति  ------>  ABHIVYAKTI



In [ ]:
line = 'अद्भुत अड्डा अविस्मरणीय श्रेय उत्कृष्ट आवृत स्तब्ध प्रत्यारोपण सामिध्य परिवर्तन परितृप्त'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

अद्भुत  ------>  ADBHUT

अड्डा  ------>  ADDA

अविस्मरणीय  ------>  AVISMARIAN

श्रेय  ------>  SHRAYA

उत्कृष्ट  ------>  UTKRISHT

आवृत  ------>  AVRIT

स्तब्ध  ------>  STABADH

प्रत्यारोपण  ------>  PRATYAROPON

सामिध्य  ------>  SAMIDHA

परिवर्तन  ------>  PARIVARTAN

परितृप्त  ------>  PARITHIPTA



In [ ]:
line = 'परिष्कृत क्रोधाग्नि वृत्ताकार क्रियाकलाप तिरस्कृत प्रतीक दिव्यद्रिष्टि परंपरा संरचना संचयिता'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

परिष्कृत  ------>  PARISHKRAT

क्रोधाग्नि  ------>  KRODHAGNI

वृत्ताकार  ------>  VRITTAKAR

क्रियाकलाप  ------>  CRIYAKALAP

तिरस्कृत  ------>  TIRASKRIT

प्रतीक  ------>  PRATIK

दिव्यद्रिष्टि  ------>  DIVYADRISTY

परंपरा  ------>  PARANPARA

संरचना  ------>  SANRACHANA

संचयिता  ------>  SANCHYITA



In [ ]:
line = 'यश तेजस शंकर अंकित श्रीजीत अनुराग शिवांश'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

यश  ------>  YASH

तेजस  ------>  TEJAS

शंकर  ------>  SHANKAR

अंकित  ------>  ANKIT

श्रीजीत  ------>  SRIJEET

अनुराग  ------>  ANURAG

शिवांश  ------>  SHIVANSH



In [ ]:
line = 'आप सभी को सूचित किया जाता है की हमारे प्रिये मित्र श्रीमान सचिन यदाव जी का विवाह फिक्स हो चुका है श्रीमान मोहित नागर अब सबसे Phle व्यक्ति नही है और हमारी भाभी का नाम श्रीमती वंशिका पटका है'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

आप  ------>  AAP

सभी  ------>  SABHI

को  ------>  KO

सूचित  ------>  SUCHIT

किया  ------>  KIYA

जाता  ------>  JAATA

है  ------>  HAI

की  ------>  KII

हमारे  ------>  HAMARE

प्रिये  ------>  PRIYE

मित्र  ------>  MITRA

श्रीमान  ------>  SREEMAAN

सचिन  ------>  SACHIN

यदाव  ------>  YADAV

जी  ------>  JEE

का  ------>  KA

विवाह  ------>  VIVAAH

फिक्स  ------>  FICKS

हो  ------>  HO

चुका  ------>  CHUKA

है  ------>  HAI

श्रीमान  ------>  SREEMAAN

मोहित  ------>  MOHIT

नागर  ------>  NAGAR

अब  ------>  AB

सबसे  ------>  SABSE

  ------>  

व्यक्ति  ------>  VYAKTI

नही  ------>  NAHI

है  ------>  HAI

और  ------>  AUR

हमारी  ------>  HAMARI

भाभी  ------>  BHABHI

का  ------>  KA

नाम  ------>  NAAM

श्रीमती  ------>  SRIMATI

वंशिका  ------>  VANSHIKA

पटका  ------>  PATKA

है  ------>  HAI



In [ ]:
stop

In [ ]:
%reset-f

# Get Saved Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os . chdir ( "/content/gdrive/My Drive/Colab Notebooks/5) Deep Learning/Projects/Common/Sign Board Text Detection")

In [ ]:
%reset-f

In [ ]:
train_model = False

In [ ]:
import torch
from torch import nn
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Re-Define Classes

In [ ]:
class self_attention (nn.Module) :
    def __init__ (self, embedding_size=512, num_heads=8) :
        super(self_attention, self).__init__()
        self.embedding_size, self.num_heads, self.head_dimensions, self.keys_matrix, self.queries_matrix, self.values_matrix, self.final_matrix = embedding_size, num_heads, embedding_size//num_heads, nn.Linear((embedding_size//num_heads), (embedding_size//num_heads), bias=False), nn.Linear((embedding_size//num_heads), (embedding_size//num_heads), bias=False), nn.Linear((embedding_size//num_heads), (embedding_size//num_heads), bias=False), nn.Linear((embedding_size//num_heads) * num_heads, embedding_size)
        assert self.head_dimensions * num_heads == embedding_size, "Embed size needs to be divisible by the number of heads"
    def forward (self, keys, queries, values, mask=None) :
        self.keys_cross_prod_queries = torch.einsum('nqhd,nkhd->nhqk', self.queries_matrix(queries.reshape(queries.shape[0], queries.shape[1], self.num_heads, self.head_dimensions)), self.keys_matrix(keys.reshape(queries.shape[0], keys.shape[1], self.num_heads, self.head_dimensions)))
        if mask is not None : self.keys_cross_prod_queries = self.keys_cross_prod_queries.masked_fill(mask == 0, float("-1e20"))
        return self.final_matrix(torch.einsum('nhql,nlhd->nqhd', torch.softmax(self.keys_cross_prod_queries/(self.embedding_size**(1/2)), dim=3), self.values_matrix(values.reshape(queries.shape[0], values.shape[1], self.num_heads, self.head_dimensions))).reshape(queries.shape[0], queries.shape[1], self.num_heads * self.head_dimensions))

In [ ]:
class encoder_cell (nn.Module) :
    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(encoder_cell, self).__init__()
        self.self_attention_block, self.normalize, self.dropout, self.feed_forward_block = self_attention(embedding_size, num_heads_in_attention_cell), nn.LayerNorm(embedding_size), nn.Dropout(dropout), nn.Sequential(nn.Linear(embedding_size, embedding_size*expanssion_size), nn.ReLU(), nn.Linear(embedding_size*expanssion_size, embedding_size))
    def add_and_concat_operation (self, matrix1, matrix2) : return self.dropout(self.normalize(matrix1 + matrix2))
    def forward (self, keys, queries, values, mask=None) : return self.add_and_concat_operation(self.feed_forward_block(self.add_and_concat_operation(self.self_attention_block(keys, queries, values, mask), queries)), self.add_and_concat_operation(self.self_attention_block(keys, queries, values, mask), queries))

In [ ]:
class Encoder (nn.Module) :
    def __init__ (self, sorce_vocab_size, device='cpu', max_input_length=16, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, num_encoder_layers=6, dropout=0) :
        super(Encoder, self).__init__()
        self.embedding_size, self.device, self.dropout, self.word_embedding, self.positional_embedding, self.layers = embedding_size, device, nn.Dropout(dropout),nn.Embedding(sorce_vocab_size, embedding_size), nn.Embedding(max_input_length, embedding_size), nn.ModuleList([encoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout).to(device) for i in range(num_encoder_layers)])
    def forward (self, word_vector, mask=None) :
        output = self.dropout(self.word_embedding(word_vector) + self.positional_embedding(torch.arange(0, word_vector.shape[1]).expand(word_vector.shape[0], word_vector.shape[1]).to(self.device)))
        for layer in self.layers : output = layer(output, output, output, mask)
        return output

In [ ]:
class sub_decoder_cell (nn.Module) :
    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(sub_decoder_cell, self).__init__()
        self.self_attention_block, self.normalize, self.dropout, self.feed_forward_block = self_attention(embedding_size, num_heads_in_attention_cell), nn.LayerNorm(embedding_size), nn.Dropout(dropout), nn.Sequential(nn.Linear(embedding_size, embedding_size*expanssion_size), nn.ReLU(), nn.Linear(embedding_size*expanssion_size, embedding_size))
    def add_and_concat_operation (self, matrix1, matrix2) : return self.dropout(self.normalize(matrix1 + matrix2))
    def forward (self, keys, queries, values, mask=None) : return self.add_and_concat_operation(self.feed_forward_block(self.add_and_concat_operation(self.self_attention_block(keys, queries, values, mask), queries)), self.add_and_concat_operation(self.self_attention_block(keys, queries, values, mask), queries))

In [ ]:
class decoder_cell (nn.Module) :
    def __init__ (self, embedding_size=512, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.) :
        super(decoder_cell, self).__init__()
        self.self_attention_block, self.normalize, self.dropout, self.sub_decoder_cell = self_attention(embedding_size, num_heads_in_attention_cell), nn.LayerNorm(embedding_size), nn.Dropout(dropout), sub_decoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout)
    def forward (self, keys, queries, values, source_mask=None, target_mask=None) : return self.sub_decoder_cell(keys, self.dropout(self.normalize(self.self_attention_block(queries, queries, queries, target_mask) + queries)), values, source_mask)

In [ ]:
class Decoder (nn.Module) :
    def __init__ (self, target_vocab_size, embedding_size=512, num_decoder_layers=6, num_heads_in_attention_cell=8, max_output_length=16, expanssion_size=2, dropout=0, device='cpu') :
        super(Decoder, self).__init__()
        self.embedding_size, self.device, self.word_embeddings, self.positional_embedding, self.layers, self.dropout, self.FC_layer = embedding_size, device, nn.Embedding(target_vocab_size, embedding_size), nn.Embedding(max_output_length, embedding_size), nn.ModuleList([decoder_cell(embedding_size, num_heads_in_attention_cell, expanssion_size, dropout) for _ in range(num_decoder_layers)]), nn.Dropout(dropout), nn.Linear(embedding_size, target_vocab_size)
    def forward (self, keys, queries, values, source_mask=None, target_mask=None) :
        output = self.dropout(self.word_embeddings(queries) + self.positional_embedding(torch.arange(0, queries.shape[1]).expand(queries.shape[0], queries.shape[1]).to(self.device)))
        for layer in self.layers : output = layer.forward(keys, output, values, source_mask, target_mask)
        return self.FC_layer(output)

In [ ]:
class Transformer (nn.Module) :
    def __init__ (self, source_vocab_size, target_vocab_size, source_pad_index=0, target_pad_index=0, embedding_size=512, num_encoder_layers=6, num_decoder_layers=6, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0., max_input_length=16, max_output_length=16, device='cpu') :
        super(Transformer, self).__init__()
        self.encoder, self.decoder, self.source_pad_index, self.target_pad_index, self.device = Encoder(source_vocab_size, device, max_input_length, embedding_size, num_heads_in_attention_cell, expanssion_size, num_encoder_layers, dropout), Decoder(target_vocab_size, embedding_size, num_decoder_layers, num_heads_in_attention_cell, max_output_length, expanssion_size, dropout, device), source_pad_index, target_pad_index, device
    def forward (self, source_vector, target_vector) : return self.decoder(keys=self.encoder(source_vector, (source_vector != self.source_pad_index).unsqueeze(1).unsqueeze(2).to(self.device)), queries=target_vector, values=self.encoder(source_vector, (source_vector != self.source_pad_index).unsqueeze(1).unsqueeze(2).to(self.device)), source_mask=(source_vector != self.source_pad_index).unsqueeze(1).unsqueeze(2).to(self.device), target_mask=torch.tril(torch.ones(target_vector.shape[0], target_vector.shape[0])).expand(target_vector.shape[0], 1, target_vector.shape[0], target_vector.shape[0]).to(self.device))

### Load Other Necessary Elements

In [ ]:
# Hindi Unicode Hex Range is 2304:2435
pad_char = '<pad>'
hindi_alphabets_dict = { pad_char:0, '<sos>':1, '<eos>':2 }
for num in range(2304, 2436) : hindi_alphabets_dict[chr(num)] = num - 2301
print(hindi_alphabets_dict)
print(len(hindi_alphabets_dict))

english_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '<pad>'
english_alphabets_dict = { pad_char:0, '<sos>':1, '<eos>':2 }
for index, alphabet in enumerate(english_alphabets) : english_alphabets_dict[alphabet] = index + 3
print(english_alphabets_dict)
print(len(english_alphabets_dict))

english_reverse_dict = {}
for keys in english_alphabets_dict : english_reverse_dict[english_alphabets_dict[keys]] = keys
print(english_reverse_dict)

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ऀ': 3, 'ँ': 4, 'ं': 5, 'ः': 6, 'ऄ': 7, 'अ': 8, 'आ': 9, 'इ': 10, 'ई': 11, 'उ': 12, 'ऊ': 13, 'ऋ': 14, 'ऌ': 15, 'ऍ': 16, 'ऎ': 17, 'ए': 18, 'ऐ': 19, 'ऑ': 20, 'ऒ': 21, 'ओ': 22, 'औ': 23, 'क': 24, 'ख': 25, 'ग': 26, 'घ': 27, 'ङ': 28, 'च': 29, 'छ': 30, 'ज': 31, 'झ': 32, 'ञ': 33, 'ट': 34, 'ठ': 35, 'ड': 36, 'ढ': 37, 'ण': 38, 'त': 39, 'थ': 40, 'द': 41, 'ध': 42, 'न': 43, 'ऩ': 44, 'प': 45, 'फ': 46, 'ब': 47, 'भ': 48, 'म': 49, 'य': 50, 'र': 51, 'ऱ': 52, 'ल': 53, 'ळ': 54, 'ऴ': 55, 'व': 56, 'श': 57, 'ष': 58, 'स': 59, 'ह': 60, 'ऺ': 61, 'ऻ': 62, '़': 63, 'ऽ': 64, 'ा': 65, 'ि': 66, 'ी': 67, 'ु': 68, 'ू': 69, 'ृ': 70, 'ॄ': 71, 'ॅ': 72, 'ॆ': 73, 'े': 74, 'ै': 75, 'ॉ': 76, 'ॊ': 77, 'ो': 78, 'ौ': 79, '्': 80, 'ॎ': 81, 'ॏ': 82, 'ॐ': 83, '॑': 84, '॒': 85, '॓': 86, '॔': 87, 'ॕ': 88, 'ॖ': 89, 'ॗ': 90, 'क़': 91, 'ख़': 92, 'ग़': 93, 'ज़': 94, 'ड़': 95, 'ढ़': 96, 'फ़': 97, 'य़': 98, 'ॠ': 99, 'ॡ': 100, 'ॢ': 101, 'ॣ': 102, '।': 103, '॥': 104, '०': 105, '१': 106, '२': 107, '३': 108, '४': 10

In [ ]:
def return_cleaned_text(word, language) :
    final_word = ''
    word = word.upper()
    if language == 'eng' :
        for letter in word :
            if letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' : final_word += letter
    else :
        for letter in word :
            if letter in 'ऀँंःऄअआइईउऊऋऌऍऎएऐऑऒओऔकखगघङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसहऺऻ़ऽािीुूृॄॅॆेैॉॊोौ्ॎॏॐ॒॑॓॔ॕॖॗक़ख़ग़ज़ड़ढ़फ़य़ॠॡॢॣ०१२३४५६७८९॰ॱॲॳॴॵॶॷॸॹॺॻॼॽॾॿঀঁংঃ' :
                final_word += letter
    return final_word

In [ ]:
def output_english_word (hindi_word, hindi_dict, english_dict, model, max_output_length=16, device=device, verbose=True) :
    predictions = [1]
    english_word = ''
    hindi_word = return_cleaned_text(hindi_word, language='hin')[:14]
    hindi_word_vector = np.array([1] + [0]*len(hindi_word) + [2] + [0]*(14-len(hindi_word)))
    for i, letter in enumerate(hindi_word) : hindi_word_vector[i+1] = hindi_dict[letter]
    hindi_word_vector = torch.tensor(np.array(hindi_word_vector).reshape(1,-1))
    for i in range(max_output_length) :
        input_tensor = torch.tensor(predictions).unsqueeze(0).to(device)
        with torch.no_grad() : out = model(hindi_word_vector.to(device), input_tensor.to(device))
        word_index = out.argmax(dim=-1)[:, -1].item()
        predictions.append(word_index)
        if word_index == 2 : break
        else : english_word += english_dict[word_index]
    if verbose : print(hindi_word, ' ------> ', english_word)
    return predictions, hindi_word

### Load Model

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/5) Deep Learning/Projects/Common/Sign Board Text Detection/Models - CheckPoints/Transliteration/Complete Model.pt'

if torch.cuda.is_available() :
    model_weights = torch.load(path)
else :
    model_weights = torch.load(path, map_location=torch.device('cpu'))

In [ ]:
model = Transformer(source_vocab_size=len(hindi_alphabets_dict), target_vocab_size=len(english_alphabets_dict), source_pad_index=0, target_pad_index=0,
                          embedding_size=512, num_encoder_layers=6, num_decoder_layers=6, num_heads_in_attention_cell=8, expanssion_size=2, dropout=0.,
                          max_input_length=16, max_output_length=16, device=device).to(device)

In [ ]:
model.load_state_dict(model_weights)

<All keys matched successfully>

In [ ]:
line = 'यश तेजस शंकर अंकित श्रीजीत अनुराग शिवांश'.split()

for hindi_word in line :
    ele = output_english_word(hindi_word, hindi_alphabets_dict, english_reverse_dict, model)
    print()

यश  ------>  YASH

तेजस  ------>  TEJAS

शंकर  ------>  SHANKAR

अंकित  ------>  ANKIT

श्रीजीत  ------>  SRIJIT

अनुराग  ------>  ANURAG

शिवांश  ------>  SHIWAANSH

